#### 고려대한국어대사전 오픈프로 포멧으로 변경 (ohw)

In [72]:
### 칼럼 설명 
# eid : 제공처 ID, ohw : 표제어, onum : 어깨번호, type : 표제어종류 
# morph : 형태, origin : 원어, (suppos, subpos) : 품사, onum.1 : 뜻번호 
# explan : 뜻풀이, technic : 전문분야, pattern : 패턴, semantic : 의미정보, restric : 꼴 정보 
# theta : 패턴 설명, example : 예문, stype : 종류, relohw : 연관어 
# prior : 우선순위 (2로 고정), conform : 활용형, contype : 부가명칭(불규칙), contype_explan : 부가설명


In [83]:
import pandas as pd
import re 

pattern_format = {'ꄍ' : '(명사)', 'ꄉ' : '(동사)', 'ꄺ' : '(형용사)', 'ꄖ' : '(부사)'}
pos_format = {'ꃃ':'명사',  'ꂽ' : '대명사', 'ꃓ' : '수사',  'ꂿ' : '동사', 'ꃰ' : '형용사', 
            'ꂴ' : '관형사', 'ꃌ' : '부사', 'ꂳ' : '감탄사', 'ꃦ' : '조사', 'ꃥ' : '접속사', 'ꃆ' : ['접두사', '접미사'], 
            'ꃁ' : ['종결 어미', '연결 어미', '전성 어미', '선어말 어미', '어미']}
# 하위 품사 표 
subpos_format = {'무', '일반', '고유', '자립', '의존', '인칭', '지시', '타', '자', '보', '활', '두', '미', '선', '종', '연', '잔'}
# 불규칙 멘트 
irregular = {'ㄷ∃' : 'ㄷ 불규칙', 'ㅂ∃' : 'ㅂ 불규칙', 'ㅅ∃' : 'ㅅ 불규칙', 'ㅎ∃' : 'ㅎ 불규칙', '러∃' : '러 불규칙', '르∃' : '르 불규칙', 
            '여∃' : '여 불규칙', '우∃' : '우 불규칙', '너라∃' : '너라 불규칙'} 

input_path = '../asset/ohw_sample.csv'
target = pd.read_csv(input_path)

In [84]:
target.fillna('', inplace=True)
target['type'] = '단어'
target['prior'] = int(2)
target['contype_explan'] = '부가설명란 ex 여 불규칙은 어쩌고'
target = target[['eid', 'ohw', 'onum', 'type', 'morph', 'origin', 'etymol', 'explan_onum',
        'suppos', 'subpos', 'explan', 'technic', 'pattern', 'semantic',
        'restric', 'theta', 'example', 'stype', 'relohw', 'prior', 'contype', 'conform', 'contype_explan']]
target = target.drop_duplicates()
target = target.reset_index(drop=True)
target

,eid,ohw,onum,type,morph,origin,etymol,explan_onum,suppos,subpos,...,semantic,restric,theta,example,stype,relohw,prior,contype,conform,contype_explan
0,1,ㄱ,0,단어,,,,1,무,무,...,,,,,,,2,,,부가설명란 ex 여 불규칙은 어쩌고
1,1,ㄱ,0,단어,,,,2,무,무,...,,,,,,,2,,,부가설명란 ex 여 불규칙은 어쩌고
2,2,ㄲ,0,단어,,,,1,무,무,...,,,,,,,2,,,부가설명란 ex 여 불규칙은 어쩌고
3,2,ㄲ,0,단어,,,,2,무,무,...,,,,,,,2,,,부가설명란 ex 여 불규칙은 어쩌고
4,3,ㄳ,0,단어,,,,1,무,무,...,,,,,,,2,,,부가설명란 ex 여 불규칙은 어쩌고
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18955,9998,개금,1,단어,[蓋金],(蓋金),,1,ꃃ,고유,...,,,,,유의,천개소문,2,,,부가설명란 ex 여 불규칙은 어쩌고
18956,9999,개금,2,단어,[改金],(改金),,1,ꃃ,무,...,,,,,참고,개금불사,2,,,부가설명란 ex 여 불규칙은 어쩌고
18957,9999,개금,2,단어,[改金],(改金),,1,ꃃ,무,...,,,,,유의,개채,2,,,부가설명란 ex 여 불규칙은 어쩌고
18958,10000,개금,3,단어,[開襟],(開襟),,1,ꃃ,무,...,,,,,유의,돕지,2,,,부가설명란 ex 여 불규칙은 어쩌고


In [85]:
# 1. pattern 폰트 변경 
LEN = len(target)
for i in range(LEN):
    if target.loc[i, 'pattern'] == '':
        continue
    for pattern in pattern_format:
        target.loc[i, 'pattern'] = re.sub(pattern, pattern_format[pattern], target.loc[i, 'pattern'])
    target.loc[i, 'pattern'] = target.loc[i, 'pattern'][1:-1]

In [86]:
cnt = 0
for i in range(LEN):
    if cnt == 10:
        break
    if target.loc[i, 'pattern'] != '':
        print(target.loc[i, 'pattern'])
        cnt += 1

(명사)이
(명사)이
(명사)이
(명사)이 (명사)을 (명사)에
(명사)이 (명사)을
(명사)이
(명사)이
(명사)이
(명사)이 (명사)에/(명사)에서
(명사)이 (명사)을 (명사)으로


In [87]:
# 2-1, 2-2. 품사 폰트 + 품사 필드 두개 (suppos, subpos) 합치기 
for i in range(LEN):
    if target.loc[i, 'suppos'] == '' or target.loc[i, 'suppos'] == '무':
        continue
    pos_symbol = target.loc[i, 'suppos']
    if pos_symbol not in pos_format:
        continue
    # 명사 
    if pos_symbol == 'ꃃ':
        if target.loc[i, 'subpos'] == '무':
            target.loc[i, 'suppos'] = '명사'
        else:
            target.loc[i, 'suppos'] = target.loc[i, 'subpos'] + '명사'
    # 대명사
    elif pos_symbol == 'ꂽ':
        if target.loc[i, 'subpos'] == '무':
            target.loc[i, 'suppos'] = '대명사'
        else:
            target.loc[i, 'suppos'] = target.loc[i, 'subpos'] + " " + '대명사'
    # 접사
    elif pos_symbol == 'ꃆ':
        if target.loc[i, 'subpos'] == '두':
            target.loc[i, 'suppos'] = '접두사'
        else: 
            target.loc[i, 'suppos'] = '접미사'
    # 동사
    elif pos_symbol == 'ꂿ':
        if target.loc[i, 'subpos'] == '무':
            target.loc[i, 'suppos'] = '동사'
        elif target.loc[i, 'subpos'] == '보':
            target.loc[i, 'suppos'] = '보조동사'
        else: 
            target.loc[i, 'suppos'] = target.loc[i, 'subpos'] + "동사"
    # 형용사 
    elif pos_symbol == 'ꃰ':
        if target.loc[i, 'subpos'] == '무':
            target.loc[i, 'suppos'] = '형용사'
        else:
            target.loc[i, 'suppos'] = '보조형용사'
    # 어미 
    elif pos_symbol == 'ꃁ':
        if target.loc[i, 'subpos'] == '무':
            target.loc[i, 'suppos'] = '어미'
        elif target.loc[i, 'subpos'] == '선':
            target.loc[i, 'suppos'] = '선어말 어미'
        elif target.loc[i, 'subpos'] == '종':
            target.loc[i, 'suppos'] = '종결 어미'
        elif target.loc[i, 'subpos'] == '연':
            target.loc[i, 'suppos'] = '연결 어미'
        elif target.loc[i, 'subpos'] == '전':
            target.loc[i, 'suppos'] = '전성 어미'
    # 준 
    elif target.loc[i, 'suppos'] == '준':
        target.loc[i, 'suppos'] = ''
    # 나머지 
    else:
        target.loc[i, 'suppos'] = pos_format[pos_symbol]

In [88]:
cnt = 0
for i in range(LEN):
    if cnt == 10:
        break
    if target.loc[i, 'suppos'] != '무' and target.loc[i, 'suppos'] != '명사':
        print(target.loc[i, 'suppos'])
        cnt += 1

조사
접속사
접속사
접속사
접속사
접속사
접속사
접속사
자동사
고유명사


In [89]:
# 2-3. "무"품사 대체 
for i in range(LEN):
    if target.loc[i, 'suppos'] == '무':
        if target.loc[i, 'explan_onum'] == 1:
            if target.loc[i, 'subpos'] == '무':
                target.loc[i, 'suppos'] = ''
            else:
                target.loc[i, 'suppos'] = target.loc[i, 'subpos'] + '명사'
        else:
            target.loc[i, 'suppos'] = target.loc[i-1, 'suppos']

target = target.drop('subpos', axis=1)

In [100]:
LEN = len(target)
extra_list = []
for i in range(LEN):
    if target.loc[i, 'example'] != '':
        ohw = target.loc[i, 'ohw']
        word = f'<b>{ohw}</b>'
        target.loc[i, 'example'] = target.loc[i, 'example'][1:].replace('~', word)
        if ' / ' in target.loc[i, 'example']:
            temps = target.loc[i, 'example'].split(' / ')
            print(temps)
            target.loc[i, 'example'] = temps[0]
            for temp in temps[1:]:
                new_row = target.loc[i, :].copy()
                new_row['example'] = temp
                extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'onum':[extra_list[i]['onum']], 
                            'type':[extra_list[i]['type']], 'morph':[extra_list[i]['morph']], 'origin':[extra_list[i]['origin']],  
                            'etymol':[extra_list[i]['etymol']], 'explan_onum':[extra_list[i]['explan_onum']], 'suppos':[extra_list[i]['suppos']], 
                            'explan':[extra_list[i]['explan']], 'technic':[extra_list[i]['technic']], 'pattern':[extra_list[i]['pattern']],
                            'semantic':[extra_list[i]['semantic']], 'restric':[extra_list[i]['restric']], 'theta':[extra_list[i]['theta']],
                            'example':extra_list[i]['example'], 'stype':[extra_list[i]['stype']], 'relohw':[extra_list[i]['relohw']],
                            'prior':[extra_list[i]['prior']], 'contype':[extra_list[i]['contype']], 'conform':[extra_list[i]['conform']], 
                            'conform_explan':[extra_list[i]['conform_explan']],})
    target = pd.concat([target, new_row], ignore_index=True)

target = target.sort_values(by='eid', kind='stable')
target = target.reset_index(drop=True)

In [101]:
# 4-1. contype 변경 
for i in range(LEN):
    contype_symbol = target.loc[i, 'contype']
    if contype_symbol == '':
        continue
    target.loc[i, 'contype'] = irregular[contype_symbol]

In [103]:
LEN = len(target)
for i in range(LEN):
    if target.loc[i, 'contype'] != '':
        print(target.loc[i, 'conform'])

<가가대소하여/가가대소해, 가가대소하니>
<가각하여/가각해, 가각하니>
<가감산하여/가감산해, 가감산하니>
<가감하여/가감해, 가감하니>
<가강하여/가강해, 가강하니>
<가거하여/가거해, 가거하니>
<가거하여/가거해, 가거하니>
<가거하여/가거해, 가거하니>
<가격하여/가격해, 가격하니>
<가격하여/가격해, 가격하니>
<가격하여/가격해, 가격하니>
<가결의하여/가결의해, 가결의하니>
<가결하여/가결해, 가결하니>
<가결하여/가결해, 가결하니>
<가결하여/가결해, 가결하니>
<가계약하여/가계약해, 가계약하니>
<가계하여/가계해, 가계하니>
<가계하여/가계해, 가계하니>
<가계하여/가계해, 가계하니>
<가계하여/가계해, 가계하니>
<가계하여/가계해, 가계하니>
<가공스러워, 가공스러우니>
<가공하여/가공해, 가공하니>
<가공하여/가공해, 가공하니>
<가공하여/가공해, 가공하니>
<가관스러워, 가관스러우니>
<가관스러워, 가관스러우니>
<가관하여/가관해, 가관하니>
<가교하여/가교해, 가교하니>
<가귀노름하여/가귀노름해, 가귀노름하니>
<가귀대기하여/가귀대기해, 가귀대기하니>
<가극하여/가극해, 가극하니>
<가금하여/가금해, 가금하니>
<가금하여/가금해, 가금하니>
<가급유하여/가급유해, 가급유하니>
<가급인족하여/가급인족해, 가급인족하니>
<가급하여/가급해, 가급하니>
<가급하여/가급해, 가급하니>
<가긍스러워, 가긍스러우니>
<가긍하여/가긍해, 가긍하니>
<가까이하여/가까이해, 가까이하니>
<가까이하여/가까이해, 가까이하니>
<가까이하여/가까이해, 가까이하니>
<가까이하여/가까이해, 가까이하니>
<가까이하여/가까이해, 가까이하니>
<가까워, 가까우니>
<가까워, 가까우니>
<가까워, 가까우니>
<가까워, 가까우니>
<가까워, 가까우니>
<가까워, 가까우니>
<가까워, 가까우니>
<가깝디가까워, 가깝디가까우니>
<가난하여/가난해, 가난하니>
<가난하여/가난해, 가난하니>
<가난하여/가난해, 가난하니>
<가난하여/가난해, 가난하니>
<가난하여/가난해, 가난하니>
<

In [104]:
# 4-2. conform 행 나누기 
# 일단 임시적으로 한 행에 나열
for i in range(LEN):
    if target.loc[i, 'explan_onum'] == "" or target.loc[i, 'explan_onum'] != 1:
        target.loc[i, ['conform', 'contype']] = ""
    else:
        conform_words = target.loc[i, 'conform'][1:-1]
        conform_words = re.sub('/', " ", conform_words)
        conform_words = re.sub(',', '', conform_words)
        target.loc[i, 'conform'] = conform_words

In [105]:
LEN = len(target)
for i in range(LEN):
    if target.loc[i, 'contype'] != '':
        print(target.loc[i, 'conform'])

가가대소하여 가가대소해 가가대소하니
가각하여 가각해 가각하니
가감산하여 가감산해 가감산하니
가감하여 가감해 가감하니
가강하여 가강해 가강하니
가거하여 가거해 가거하니
가거하여 가거해 가거하니
가격하여 가격해 가격하니
가결의하여 가결의해 가결의하니
가결하여 가결해 가결하니
가결하여 가결해 가결하니
가결하여 가결해 가결하니
가계약하여 가계약해 가계약하니
가계하여 가계해 가계하니
가계하여 가계해 가계하니
가계하여 가계해 가계하니
가계하여 가계해 가계하니
가계하여 가계해 가계하니
가공스러워 가공스러우니
가공하여 가공해 가공하니
가공하여 가공해 가공하니
가관스러워 가관스러우니
가관하여 가관해 가관하니
가교하여 가교해 가교하니
가귀노름하여 가귀노름해 가귀노름하니
가귀대기하여 가귀대기해 가귀대기하니
가극하여 가극해 가극하니
가금하여 가금해 가금하니
가금하여 가금해 가금하니
가급유하여 가급유해 가급유하니
가급인족하여 가급인족해 가급인족하니
가급하여 가급해 가급하니
가급하여 가급해 가급하니
가긍스러워 가긍스러우니
가긍하여 가긍해 가긍하니
가까이하여 가까이해 가까이하니
가까워 가까우니
가깝디가까워 가깝디가까우니
가난하여 가난해 가난하니
가난하여 가난해 가난하니
가난하여 가난해 가난하니
가난하여 가난해 가난하니
가난하여 가난해 가난하니
가납하여 가납해 가납하니
가납하여 가납해 가납하니
가납하여 가납해 가납하니
가년스러워 가년스러우니
가년스러워 가년스러우니
가년하여 가년해 가년하니
가녈가녈하여 가녈가녈해 가녈가녈하니
가녈하여 가녈해 가녈하니
가녈하여 가녈해 가녈하니
가느다래 가느다라니
가느다래 가느다라니
가느다래 가느다라니
가느대 가느다니
가느스름하여 가느스름해 가느스름하니
가늠하여 가늠해 가늠하니
가늣하여 가늣해 가늣하니
가능하여 가능해 가능하니
가닐가닐하여 가닐가닐해 가닐가닐하니
가닐가닐하여 가닐가닐해 가닐가닐하니
가닐가닐하여 가닐가닐해 가닐가닐하니
가다리하여 가다리해 가다리하니
가닥가닥하여 가닥가닥해 가닥가닥하니
가닥가닥하여 가닥가닥해 가닥가닥하니
가닥스러워 가닥스러우니
가담

In [106]:
LEN = len(target)
for i in range(LEN):
    if 'EID' in target.loc[i, 'explan']:
        print(target.loc[i, 'explan'])

‘가게&EID=000134;’의 원래 말.
☞가감판&EID=000110;(加減瓣).
‘가볍다&EID=001759;’의 방언(강원, 평북, 함남).
‘홍역&EID=376390;(紅疫)’의 방언(경남).
‘개구리&EID=009918;’의 방언(제주).
이전에, ‘한글&EID=363077;’을 달리 이르던 말. 한글의 반절본문(反切本文)이 ‘가, 갸’로 시작되는 데에서 나온 말이다.
이전에, ‘한글&EID=363077;’을 달리 이르던 말. ‘가, 갸, 거, 겨, …’로 시작되는 반절본문의 첫 두 글자를 딴 것이다.
‘한글날&EID=363080;’의 이전 말.
‘가오리연&EID=002731;(---鳶)’의 방언(제주).
‘가랑이&EID=000966;’의 방언(경북).
☞가구&EID=000340;(佳句).
☞가귀대기&EID=000370;.
‘가랑비&EID=000964;’의 방언(경남).
‘가까이&EID=000431;’의 방언(평북).
‘가까이&EID=000431;’의 방언(평북).
‘가깝다&EID=000435;’의 방언(평북).
‘어서&EID=220228;’의 방언(함남).
‘갑시다&EID=008209;’의 방언(제주).
‘아내&EID=208124;’의 방언(제주).
‘갈까마귀&EID=006063;’의 방언(함북).
‘까마귀&EID=052889;’의 방언(제주).
☞가요성&EID=002776;(@可撓性).
‘겨누다&EID=016438;’의 방언(경남, 전라).
☞가느다랗다&EID=000510;.
‘거짓 갈비뼈&EID=013508;’의 이전 말.
‘가는잎송진내&EID=000563;(---松津-)’의 이전 말.
‘가는끈적살&EID=000532;’의 이전 말.
☞가느다랗다&EID=000510;.
‘갓난아이&EID=008337;’의 방언(경상).
‘가르마&EID=001265;’의 방언(경북).
☞간힘&EID=005912;.
☞가닥&EID=000627;.
☞가타카나&EID=003764;(katakana).
☞가다랑어&EID=000618;.
‘가닥&EID=000627;’의 방언

In [108]:
target['naver_id'] = ""
target = target[['naver_id', 'eid', 'ohw', 'onum', 'type', 'morph', 'origin', 'etymol',
        'explan_onum', 'suppos', 'explan', 'technic', 'pattern', 'semantic',
        'restric', 'theta', 'example', 'stype', 'relohw', 'prior', 'contype',
        'conform', 'contype_explan']]

In [111]:
OHW_ID = pd.read_csv('../asset/ohw_id.csv')
OHW_ID = OHW_ID.sort_values(by=['kordic_eid'])
OHW_ID = OHW_ID.reset_index(drop=True)
OHW_ID

,naver_ID,type,ohw,shoulder,kordic_eid
0,33486ae3f7334c4f906292fb1b331fa4,단어,고아사,0,0
1,9f65f6c2095949c7bb2b0f30e71bfe5d,단어,요청서,0,0
2,0ff01bf7910345c0a81fd576ba1187fc,단어,요잡하다,0,0
3,9547b0abde7f4a62b87efec87ab3223c,단어,요요하다,7,0
4,c9c3a1a130c44e3b9ff4713f2ea2774e,단어,요요하다,6,0
...,...,...,...,...,...
394990,4a78e0ed0a154de491667a136cd31dcc,단어,삼경사서석의,0,466224
394991,4031a65b1c22425cb0db13314e8a1c08,단어,완력시세,0,466254
394992,3366ae9f4b6e471bb78a40f730ed42bd,단어,튜브,0,466285
394993,c474f86b32a645ccb14fb632b5700be1,단어,턱,4,466286


In [ ]:
# 5.2 최종 중복처리 
condition = target.duplicated(subset=['eid'])
target.loc[condition, 'eid':'etymol'] = ''
target

,eid,ohw,onum,type,morph,origin,etymol,explan_onum,suppos,explan,...,semantic,restric,theta,example,stype,relohw,prior,contype,conform,contype_explan
0,1,ㄱ,0,단어,,,,1,,"한글 자모의 첫째 글자. 이름은 ‘기역’이다. ‘가’에서와 같이 초성자로, ‘악’에...",...,,,,,,,2,,,부가설명란 ex 여 불규칙은 어쩌고
1,,,,,,,,2,,자음의 하나. 목젖으로 콧길을 막고 혀뿌리를 입천장에 붙여 날숨을 막았다가 뗄 때 ...,...,,,,,,,2,,,부가설명란 ex 여 불규칙은 어쩌고
2,2,ㄲ,0,단어,,,,1,,한글 자모 ‘ㄱ’을 어울러 쓴 글자. 이름은 ‘쌍(雙)기역’이다. ‘깨’에서와 같이...,...,,,,,,,2,,,부가설명란 ex 여 불규칙은 어쩌고
3,,,,,,,,2,,자음의 하나. 목젖으로 콧길을 막고 후두 근육을 긴장한 상태로 혀뿌리를 입천장에 붙...,...,,,,,,,2,,,부가설명란 ex 여 불규칙은 어쩌고
4,3,ㄳ,0,단어,,,,1,,한글 자모 ‘ㄱ’과 ‘ㅅ’을 어울러 쓴 글자. 이름은 ‘기역시옷’이다. ‘몫’에서와...,...,,,,,,,2,,,부가설명란 ex 여 불규칙은 어쩌고
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18955,,,,,,,,,,,...,,,,,유의,천개소문,2,,,부가설명란 ex 여 불규칙은 어쩌고
18956,9999,개금,2,단어,[改金],(改金),,1,명사,불상(佛像)에 새로 금칠을 함.,...,,,,,참고,개금불사,2,,,부가설명란 ex 여 불규칙은 어쩌고
18957,,,,,,,,,,,...,,,,,유의,개채,2,,,부가설명란 ex 여 불규칙은 어쩌고
18958,10000,개금,3,단어,[開襟],(開襟),,1,명사,가슴속에 품은 생각을 털어놓음.,...,,,,,유의,돕지,2,,,부가설명란 ex 여 불규칙은 어쩌고


In [ ]:
LEN = len(target)
for i in range(LEN):
    if target.loc[i, 'example'] != '':
        print(target.loc[i, 'ohw'], target.loc[i, 'example'])

ㄱㄴ순 ¶개학 첫날 선생님은 ~으로 반 아이들의 이름을 부르셨다.
ㄱ자 ¶옛날 우리집은 ~로 지은 한옥이었다.
가 ¶어머니께서는 꽃밭 ~에 과꽃을 심으셨다. / 얘야, 떨어질 염려가 있으니 ~에 앉지 말고 가운데로 앉거라. / 우리 학교 운동장 ~에는 은행나무가 빙 둘러 심어져 있다.
 ¶꿀을 뜰 때에는 병 ~에 꿀을 흘리지 않도록 주의해라.
 ¶이 안건은 ~가 15표, 부가 22표로 결국에는 부결되었다.
 ¶상철이가 국어 성적은 수를 받았지만 사회 성적은 ~를 받았다.
 ¶이 영화는 논란 끝에 연소자 관람 ~의 판정을 받았다.
가 ¶아기~ 방긋 웃는다. / 철수~ 유리창을 깼다. / 그~ 그녀에게 책을 주었다. / 오늘은 은아~ 밥을 살 것이다.
 ¶동생은 마침내 의사~ 되었다. / 그녀는 이번에 엄마~ 되었다. / 이 애벌레가 자라면 나비~ 됩니다. / 친한 후배가 올해 변호사~ 되었다.
 ¶그는 바보~ 아니다. / 이제부터 너는 내 제자~ 아니다. / 그녀는 친구~ 아니라 제 조카입니다. / 유리창을 깬 사람은 그 아이~ 아니다.
 ¶도대체~ 말이 안 된다. / 옆집 아이가 하는 말은 대개~ 거짓말이다. / 이제부터~ 시작이라고 생각합니다.
 ¶나는 철수~ 보고 싶다. / 나는 학교~ 가기 싫다. / 한국 사람들에게는 라틴어~ 배우기 어렵다.
 ¶날씨가 춥지~ 않다. / 오늘따라 기분이 좋지~ 않다. / 감기 때문에 목소리가 나아지지~ 않는다. / 그에게 일을 시켰지만 영 미덥지~ 못하다.
 ¶탁자~ 너무 작구나. / 접시~ 매우 더럽다. / 중고차~ 생각보다 비싸군요. / 의자~ 낡아서 앉기 어려워요.
 ¶해~ 뜨다 / 땅거미~ 깔렸다. / 안개~ 자욱이 끼었다. / 비~ 며칠째 계속 내린다.
 ¶요리~ 완성되었다. / 컴퓨터~ 모두 도서관으로 옮겨졌다. / 민수~ 아이들에게 맞았다. / 당선자에게 상금과 상패~ 수여되었다.
 ¶아기~ 아파요. / 내~ 다소 흥분하였다. / 그~ 고향을 그리워하고 있다. / 어려운 선택을 앞두고 그녀~ 혼란에 빠졌다.
